In [1]:
#mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls

Mounted at /content/drive
drive  sample_data


In [2]:
# move into project directory
repo_name = "crop-damage-classification"
%cd /content/drive/MyDrive/Personal-Projects/$repo_name
!ls

/content/drive/MyDrive/Personal-Projects/crop-damage-classification
common	     dataloading  Index.ipynb  preprocess	     run.yaml
config.yaml  experiments  index.py     project-structure.md  transforms
data	     Index_bc.py  models       README.md	     visualization


In [3]:
# set up environment
# comment if not required
'''
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install matplotlib numpy pandas pyyaml opencv-python
'''

'\n!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118\n!pip install matplotlib numpy pandas pyyaml opencv-python\n'

# Following cells are for downloading data

In [4]:
# this cell is for downloading data.
# as of yet data is not hosted and is available in the private data folder
# comment if not required
!pip install boto3
!pip install tqdm

In [5]:
# setup some imports
#custom imports
from transforms.transforms import ToTensor, Resize, CenterCrop
from dataloading.dataset import CropDataset
from common.utils import get_exp_params, init_config, get_config, save2config
from models.resnet18 import Resnet18
from experiments.experiments import Experiment
from visualization.visualization import Visualization

#py imports
import random
import numpy as np
import os
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
import boto3
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config
from tqdm.notebook import tqdm

def get_file_folders(s3_client, bucket_name, prefix=""):
    file_names = []
    folders = []

    default_kwargs = {
        "Bucket": bucket_name,
        "Prefix": prefix
    }
    next_token = ""

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        response = s3_client.list_objects_v2(**updated_kwargs)
        contents = response.get("Contents")

        for result in contents:
            key = result.get("Key")
            if key[-1] == "/":
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get("NextContinuationToken")

    return file_names, folders

def download_files(s3_client, bucket_name, local_path, file_names, folders):
    local_path = Path(local_path)

    for folder in tqdm(folders):
        folder_path = Path.joinpath(local_path, folder)
				# Create all folders in the path
        folder_path.mkdir(parents=True, exist_ok=True)

    for file_name in tqdm(file_names):
        file_path = Path.joinpath(local_path, file_name)
				# Create folder for parent directory
        file_path.parent.mkdir(parents=True, exist_ok=True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )

data_path = 'data/input/images'
if not(os.path.exists(os.path.join(os.getcwd(), data_path))):
    client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
    file_names, folders = get_file_folders(client, 'cgiar-crop-damage-classification-challenge')
    download_files(
        client,
        'cgiar-crop-damage-classification-challenge',
        "/content/drive/MyDrive/Personal-Projects/crop-damage-classification/data/input",
        file_names,
        folders
    )

In [8]:
# initialize directories and config data
init_config()
config = get_config()
print('Config parameters\n')
print(config)

Config parameters

{'X_key': 'image', 'data_dir': '/content/drive/MyDrive/Personal-Projects/crop-damage-classification/data', 'img_dir': '/content/drive/MyDrive/Personal-Projects/crop-damage-classification/data/input/images', 'root_dir': '/content/drive/MyDrive/Personal-Projects/crop-damage-classification', 'use_gpu': False, 'y_key': 'label'}


In [9]:
# read experiment parameters
exp_params = get_exp_params()
print('Experiment parameters\n')
print(exp_params)

Experiment parameters

{'transform': {'resize_dim': 256, 'crop_dim': 224}, 'train': {'shuffle_data': True, 'batch_size': 128, 'val_split_method': 'k-fold', 'k': 2, 'val_percentage': 20, 'loss': 'cross-entropy', 'batch_interval': 512, 'epoch_interval': 1, 'num_epochs': 1}, 'model': {'name': 'resnet18', 'optimizer': 'Adam', 'lr': 0.001, 'weight_decay': 1e-05, 'amsgrad': False, 'momentum': 0.9}, 'test_model': False}


In [10]:
#initialize randomness seed
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [11]:
#preprocess data or load preprocessed data

#build label dict
label_dict = {
    'DR': 0,
    'G': 1,
    'ND': 2,
    'WD': 3,
    'other': 4
}

class_dict = {
    0: 'DR',
    1: 'G',
    2: 'ND',
    3: 'WD',
    4: 'other'
}

In [12]:
#save X_key and y_key
save2config('X_key', 'image')
save2config('y_key', 'label')

#transform data
data_transforms = transforms.Compose([ToTensor(), Resize(exp_params['transform']['resize_dim']), CenterCrop(exp_params['transform']['crop_dim'])])

#convert to dataset
ftr_dataset = CropDataset('input/Train.csv', label_dict, False, transforms=data_transforms)
test_dataset = CropDataset('input/Test.csv', label_dict, True, transforms=data_transforms)
smlen = int(0.05 * len(ftr_dataset))
smftr_dataset = torch.utils.data.Subset(ftr_dataset, list(range(smlen)))
print('Full train dataset length:', len(ftr_dataset))
print('Test dataset length:', len(test_dataset))
print('Subset train dataset length:', smlen, '\n')



Full train dataset length: 26068
Test dataset length: 8663
Subset train dataset length: 1303 



In [13]:
#model import

if exp_params['model']['name'] == 'resnet18':
    model = Resnet18(5, False)
else:
    raise SystemExit("Error: Invalid model name passed! Check run.yaml")


In [ ]:
#running experiment on small subset of the dataset
exp = Experiment(model, smftr_dataset)
model_info = exp.train()
print("\nModel validation results")

#visualization results
vis = Visualization(model_info)
vis.get_results()

Running split 0
	Running Epoch 0
		Running through training dataset


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
'''
#model training on full dataset
exp = Experiment(model, ftr_dataset)
model_info = exp.train()
print("\nModel validation results")

#visualization results
vis = Visualization(model_info)
vis.get_results()
'''

'\n#model training on full dataset\nexp = Experiment(model, ftr_dataset)\nmodel_info = exp.train()\nprint("\nModel validation results")\n\n#visualization results\nvis = Visualization(model_info)\nvis.get_results()\n'

In [14]:
#running experiment on small subset of the dataset
exp = Experiment(model, smftr_dataset)

In [ ]:
#model testing
print("Testing Best Model")
exp.test(model, test_dataset, class_dict)
#print("\nTesting Last Model")
#exp.test(model_info["last_model"], test_dataset, label_dict)

[['ID_5ZBLI5', 0.0, 0.9999998807907104, 1.289611901711396e-07, 8.137104561293157e-32, 0.0], ['ID_0W8L81', 0.0, 1.0, 6.97581853392748e-11, 1.3328226135671206e-39, 0.0], ['ID_7KIFNB', 0.0, 1.0, 4.401809472476259e-11, 9.291855234221197e-36, 0.0], ['ID_V85NEM', 0.0, 0.9999940395355225, 6.001388101140037e-06, 5.31137286472139e-24, 0.0], ['ID_8PRUL0', 0.0, 0.030822237953543663, 0.969177782535553, 5.892626294990713e-28, 0.0], ['ID_2E9C9E', 0.0, 0.0009988282108679414, 0.9990012049674988, 1.2091804448658847e-41, 0.0], ['ID_O4E311', 0.0, 1.0, 2.1228234692001816e-14, 1.187684526423142e-40, 0.0], ['ID_HTB1IE', 0.0, 1.0, 5.060707102401245e-14, 1.8864560186433552e-40, 0.0], ['ID_N1XDCQ', 1.401298464324817e-45, 1.0, 4.2170475444969213e-10, 2.998112708908703e-25, 0.0], ['ID_6GWGQ7', 2.6211427905042136e-40, 0.0058178408071398735, 0.9941821694374084, 5.228604516044625e-18, 0.0], ['ID_G36TYN', 0.0, 1.0, 1.5499000398529715e-15, 2.5405541158208933e-41, 0.0], ['ID_UBY3H7', 0.0, 1.0, 2.9758634894131147e-15, 

<ipython-input-7-7fb0fd636692>:228: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[['ID_DQDDZM', 0.0, 3.4235891175848584e-16, 1.0, 9.04448615749799e-39, 0.0], ['ID_5G69BH', 0.0, 0.0024936373811215162, 0.9975064396858215, 6.123388698786513e-32, 0.0], ['ID_RH3LDJ', 0.0, 3.583090801839717e-05, 0.9999641180038452, 8.388014242369395e-28, 0.0], ['ID_C8AYFQ', 0.0, 1.0, 3.5921782881587205e-08, 4.369600922861564e-35, 0.0], ['ID_YBGUBE', 0.0, 0.00021242140792310238, 0.9997876286506653, 5.05363985448458e-34, 0.0], ['ID_G09SP6', 0.0, 1.0, 3.7812908360024267e-10, 5.207060801147884e-25, 0.0], ['ID_57P05Q', 0.0, 1.0, 4.540118931159589e-18, 3.080281008929069e-34, 0.0], ['ID_RIB217', 0.0, 1.1383156223176982e-10, 1.0, 1.3795235828633673e-32, 0.0], ['ID_U02QS2', 0.0, 6.153822340593251e-08, 0.9999998807907104, 2.581191771286313e-42, 0.0], ['ID_22CDBM', 0.0, 2.6915770003072637e-23, 1.0, 2.059908742557481e-43, 0.0], ['ID_ZNK7TX', 0.0, 1.0, 4.3966263962857965e-10, 6.223057101551208e-32, 0.0], ['ID_NXB2CY', 0.0, 1.0, 5.49090817258957e-09, 2.021760779678821e-36, 0.0], ['ID_ELKLRL', 0.0, 1.0

<ipython-input-7-7fb0fd636692>:228: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[['ID_AMKXYB', 0.0, 1.0, 7.741002341221529e-16, 2.5464395693710576e-41, 0.0], ['ID_JBTISD', 0.0, 1.0, 5.533155060028676e-15, 1.3274257087135585e-37, 0.0], ['ID_QLETWS', 0.0, 8.977193988179977e-14, 1.0, 5.8714405655209835e-43, 0.0], ['ID_100RRK', 0.0, 0.9876987934112549, 0.012301197275519371, 9.911668449113708e-32, 0.0], ['ID_YCXVE9', 0.0, 1.6489525478391442e-06, 0.9999983310699463, 5.2117811475895e-25, 0.0], ['ID_KH9CVS', 0.0, 0.3321140706539154, 0.667885959148407, 4.5552827538109994e-38, 0.0], ['ID_WH1HYN', 0.0, 1.0, 1.7608706376695538e-13, 1.9126046138947113e-34, 0.0], ['ID_87NJ26', 0.0, 0.9999995231628418, 5.239264737610938e-07, 9.160848580677059e-41, 0.0], ['ID_YECLVH', 1.313717310304516e-41, 0.9999668598175049, 1.4875470646935618e-11, 3.318633389426395e-05, 0.0], ['ID_VSDOBR', 0.0, 0.17136865854263306, 0.8286313414573669, 4.722950357145007e-40, 0.0], ['ID_Y4ORKU', 0.0, 3.6139360304332513e-07, 0.9999996423721313, 4.203895392974451e-45, 0.0], ['ID_JAYC0L', 0.0, 1.0, 5.05928415481249

...

[['ID_QYSXKB', 0.0, 1.0, 5.938245806191844e-08, 0.0, 0.0], ['ID_XV494P', 0.0, 1.0, 6.916042738502881e-11, 4.820151319745253e-34, 0.0], ['ID_FZ906C', 0.0, 1.0, 7.728517207927898e-17, 1.1526746119494802e-17, 0.0], ['ID_DKLLYS', 0.0, 0.44246333837509155, 0.5575366616249084, 6.304628780264432e-36, 0.0], ['ID_YORPO1', 0.0, 1.5472044478936908e-10, 1.0, 2.802596928649634e-44, 0.0], ['ID_391AP7', 0.0, 1.6893905922188424e-05, 0.9999830722808838, 1.415217230053796e-33, 0.0], ['ID_MFCQHX', 0.0, 3.1129843591770623e-07, 0.9999996423721313, 0.0, 0.0], ['ID_6ZW5G1', 0.0, 0.9955109357833862, 0.00448913499712944, 1.9966919649363956e-39, 0.0], ['ID_12W3GL', 0.0, 0.7251805663108826, 0.27481943368911743, 2.781532021669263e-32, 0.0], ['ID_SENB6Z', 0.0, 1.0, 2.7955188164341394e-10, 1.5213670977342704e-37, 0.0], ['ID_9Y1ED6', 0.0, 1.1047516679241198e-08, 1.0, 8.244479498439075e-28, 0.0], ['ID_BCU44K', 0.0, 1.1626757222243106e-27, 1.0, 1.401298464324817e-45, 0.0], ['ID_BK1CQX', 0.0, 0.9999805688858032, 1.9406

<ipython-input-7-7fb0fd636692>:228: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


[['ID_VFZCUX', 0.0, 1.0, 1.069671462516908e-09, 6.980241143266846e-25, 0.0], ['ID_9JI4L7', 0.0, 1.0, 1.3463283266013093e-10, 1.401298464324817e-45, 0.0], ['ID_6PLMB1', 0.0, 6.970831600483507e-05, 0.9999302625656128, 2.382207389352189e-44, 0.0], ['ID_NLUF6C', 0.0, 0.0037901068571954966, 0.9962098598480225, 0.0, 0.0], ['ID_M1SHHC', 0.0, 4.38294733839939e-09, 1.0, 8.269022357699074e-34, 0.0], ['ID_VG80BE', 0.0, 0.001304327743127942, 0.9986956715583801, 3.682909439520056e-37, 0.0], ['ID_OKON3N', 0.0, 5.1811861467854214e-09, 1.0, 8.360752243036484e-34, 0.0], ['ID_35SIUI', 0.0, 1.0, 2.1739680808850487e-15, 5.277308786961718e-33, 0.0], ['ID_4HV5YG', 0.0, 2.2173955471771478e-07, 0.9999997615814209, 2.1274653415225805e-40, 0.0], ['ID_YJPUKP', 0.0, 0.9999896287918091, 1.037717993312981e-05, 4.194632810125264e-40, 0.0], ['ID_QUFR4S', 0.0, 3.224285194391996e-08, 1.0, 0.0, 0.0], ['ID_9I871E', 0.0, 1.0, 5.580723159797057e-15, 2.688430856409819e-20, 0.0], ['ID_WMP9MY', 0.0, 0.9999487400054932, 5.1216

<ipython-input-7-7fb0fd636692>:228: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  results_df = pd.concat([results_df, batch_df], 0)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceb

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-7734291ba897>", line 3, in <cell line: 3>
    exp.test(model, test_dataset, class_dict)
  File "<ipython-input-7-7fb0fd636692>", line 216, in test
    for bi, batch in enumerate(test_loader):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 674, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    d